### Preparing

In [1]:
# Import libraries and own functions
import os
import shutil
import re
import pandas as pd
import win32con, win32api, os

from save_data import save_csv, save_xlsx

# Files & folder names
# server-path
main_path = 'Z:\\Projects\\РФФИ-Норвегия\\Данные\\Экспериментальные данные в лаборатории\\Droplet freezing facility data\\Generating droplets'

# server-like folder path
loc_path = 'C:\\Users\\M.Vulf\\Desktop\\_Generating droplets'

# new data folder. !!! CHANGE THIS for NEW FOLDER
new_data_path = 'C:\\Users\\M.Vulf\\Desktop\\_parametric_experiment'

# new protocol name. !!! CHANGE THIS for NEW PROTOCOL
prot_name = 'Protocol_Parametric map.xlsx'

# folder for saving videos
video_folder = 'VIDEO_DATA'

# Generating data name
data_name = 'GENERATING_DATA.csv'

In [2]:
# Import & preparing dataframes
prot_df = pd.read_excel(os.path.join(new_data_path, prot_name))
main_df = pd.read_csv(os.path.join(main_path, data_name))
# get addititional df
adit_df = pd.read_excel(os.path.join(new_data_path, prot_name), sheet_name=1)
# rename columns
prot_df.columns = ['test', 'test_type', 'voltage', 'pressure',
       'impulse_time','impulse_count', 'needle', 'nozzle', 'distance',
       'calibrat_test', 'record_rate', 'comments', 'experiment_date',
       'tester']


last_test_num = main_df['test'].max()
print('Last test: #', last_test_num)
# data info
print(main_df.info())
print(prot_df.info())
print(adit_df.info())

Last test: # 278
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 713 entries, 0 to 712
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   test                      713 non-null    int64  
 1   crop                      713 non-null    int64  
 2   test_type                 713 non-null    object 
 3   voltage                   705 non-null    float64
 4   pressure                  705 non-null    float64
 5   impulse_time              705 non-null    float64
 6   needle                    705 non-null    object 
 7   nozzle                    705 non-null    object 
 8   distance                  0 non-null      float64
 9   calibrat_test             705 non-null    float64
 10  record_rate               705 non-null    float64
 11  comments                  127 non-null    object 
 12  experiment_date           713 non-null    object 
 13  tester                    713 non-null    object

C:\Users\M.Vulf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [3]:
# CREATE DICTIONARY OF FILES WITH TEST NUM
# pattern of test-files
pattern = re.compile('\d*_\w*.\w*') 
# list of selected files
selected_files_dict = {}
# get names dictionary
for root, dirs, files in os.walk(new_data_path):
    for file in files:
        research_result = re.match(pattern, file)
        if research_result:
            # get it's number
            number = int(research_result[0].split('_')[0])
            # add file to dict
            if not number in selected_files_dict:
                selected_files_dict[number] = [research_result[0]]
            else:
                selected_files_dict[number].append(research_result[0])

# RENAME TESTS AND COPY FILES
def rename_tests(row):
    # Change test name & calibr name
    prot_num = row['test']
    row['test'] = prot_num + last_test_num
    if row['calibrat_test']:
        row['calibrat_test'] = row['calibrat_test'] + last_test_num
    # Copy video with new name
    for file in selected_files_dict[prot_num]:
        source = os.path.join(new_data_path, file)
        new_name = "{:04d}_".format(row['test']) + '_'.join(file.split('_')[1:])
        destination = os.path.join(loc_path, video_folder, new_name)
        # print(source, ":", destination)
        shutil.copyfile(source, destination)
    return row

prot_df = prot_df.apply(rename_tests, axis=1)

In [8]:
# SAVE RESULT
prot_df['experiment_date'] = prot_df['experiment_date'].dt.date
prot_df.to_excel(os.path.join(loc_path, '2021-11-16_Protocol_Parametric map.xlsx'), index=False)